In [12]:
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
    AIMessage
)
from langchain_core.runnables import RunnableLambda
from langgraph.prebuilt import ToolNode
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph, START
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

from langchain_ollama import ChatOllama
from langchain_groq import ChatGroq
from langchain_openai import AzureChatOpenAI
from llama_index.llms.gemini import Gemini

import functools
import operator
from pydantic import BaseModel
from typing import Annotated, Sequence, TypedDict, Literal, List



import os
from dotenv import load_dotenv
load_dotenv(os.path.join('../config/','.env'))  


True

In [13]:
conversation_states = ['Initial','Exploring','Probing','Concluding']
members = ['convo_state_tracker', 'generate_assessments']
options = ["FINISH"] + members
','.join(options)

'FINISH,convo_state_tracker,generate_assessments'

In [14]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str

In [15]:
# llm = ChatGroq(
#     model="llama3-groq-8b-8192-tool-use-preview",
#     # model="llama-3.1-8b-instant",
#     temperature=0.0,
#     max_retries=2,
# )

# llm = ChatOllama(
#     model='llama3-groq-tool-use:latest',
#     # model='llama3.1',
#     temperature=0.0,
# )

# llm = AzureChatOpenAI(
#             api_key='6060de1a99394ebda50e0ecb1258883b',
#             api_version='2024-05-01-preview',
#             azure_endpoint='https://openai-pragateesh.openai.azure.com/',
#             deployment_name='gpt-4'
#         )   

llm = Gemini()

In [16]:
def create_supervisor_agent(llm, tools):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are helpful assistant, who can perform required task"
                "You are the best Socratic tutor, guiding the user towards understanding their own errors or misconceptions or in learning a new concept."
                "Your role:"
                "    Questioning: Ask probing questions to challenge the user's assumptions and encourage deeper thinking."
                "    Clarification: Request clarification when the user's responses are unclear or contradictory."
                "    Counter-arguments: Present counter-arguments to the user's claims to help them identify flaws in their reasoning."
                "    Guidance: Provide hints or suggestions to nudge the user towards the correct understanding."
                "    New Concepts: If learning new concepts, then list some related concepts to the given concept and ask the user whether he knows it or not. "
                "    Based on his existing knowledge, ask questions on the concpets he knows and converge on the new concept."
                "First of all when you get a message, follow these step:"
                "   1. Get the state of the current conversation using one of the worker"
                "   2. Once you got the state if it's initial generate an assessment for the user with 3-5 questions regarding the topic provide by user, using one of the worker."
                "   3. Evaluate the user based on his answers to the assessment generated."
                "   4. Once you have evaluated his performance, then start to ask questions using one of the workers by passing the state and chat history to one of the worker."
                "   5. Repeat the same process, until the user is satisfied in learning a topic or explicitly asked to change to new topic by the user."
                "Focus:"
                "    Concept understanding: Help the user grasp the underlying concepts and principles."
                "    Error identification: Assist the user in recognizing and correcting their mistakes."
                "    Critical thinking: Encourage the user to think critically and evaluate their own arguments."
                "Remember: Your primary goal is to facilitate learning, not to provide answers. "
                "By asking thought-provoking questions, you can help the user develop a deeper understanding of the topic and improve their problem-solving skills."

                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_tools(tools)

In [17]:
# system_prompt = (
#     "You are helpful assistant, who can perform required task"
#     "You are the best Socratic tutor, guiding the user towards understanding their own errors or misconceptions or in learning a new concept."
#     "Your role:"
#     "    Questioning: Ask probing questions to challenge the user's assumptions and encourage deeper thinking."
#     "    Clarification: Request clarification when the user's responses are unclear or contradictory."
#     "    Counter-arguments: Present counter-arguments to the user's claims to help them identify flaws in their reasoning."
#     "    Guidance: Provide hints or suggestions to nudge the user towards the correct understanding."
#     "    New Concepts: If learning new concepts, then list some related concepts to the given concept and ask the user whether he knows it or not. "
#     "    Based on his existing knowledge, ask questions on the concpets he knows and converge on the new concept."
#     "First of all when you get a message, follow these step:"
#     "   1. Get the state of the current conversation using one of the worker"
#     "   2. Once you got the state if it's initial generate an assessment for the user with 3-5 questions regarding the topic provide by user, using one of the worker."
#     "   3. Evaluate the user based on his answers to the assessment generated."
#     "   4. Once you have evaluated his performance, then start to ask questions using one of the workers by passing the state and chat history to one of the worker."
#     "   5. Repeat the same process, until the user is satisfied in learning a topic or explicitly asked to change to new topic by the user."
#     "Focus:"
#     "    Concept understanding: Help the user grasp the underlying concepts and principles."
#     "    Error identification: Assist the user in recognizing and correcting their mistakes."
#     "    Critical thinking: Encourage the user to think critically and evaluate their own arguments."
#     "Remember: Your primary goal is to facilitate learning, not to provide answers. "
#     "By asking thought-provoking questions, you can help the user develop a deeper understanding of the topic and improve their problem-solving skills."
#     "You have access to the following workers:"
#     "{options}"
# )


In [18]:
@tool #convo_state_tracker
def convo_state_tracker(messages: AgentState):
    """A tool, that can analyze the current conversation and return from one of the values: ['Initial','Exploring','Probing','Concluding']"""
    prompt = f"""
        system,
        Analyze the messages:
        {messages}
        Determine the current conversation state in Socratic learning Method to decide what to do next. 
        Consider factors such as the topic, depth of discussion, and user engagement. 
        Return one from this List [Initial,Exploring,Probing,Concluding]
        Respond with only on of the possible states and explanation for each states are as follows:
            - Initial -> meaning that the user is at The beginning stage of the conversation.
            - Exploring -> meaning that the user is at an In-depth discussion and exploration of topics.
            - Probing -> meaning that the user is Asking deeper questions to uncover more information.
            - Concluding -> meaning that the user is at Wrapping up the conversation and reaching a conclusion.
        Example for each state value:
            - query: 'Hi, I'm new to Machine Learning, where should I start?' 
            return: 'Initial'
            - query: 'Can you explain the difference between supervised and unsupervised learning?' 
            return: 'Exploring'
            - query: 'What happens if we use a high learning rate in training?' 
            return: 'Probing'
            - query: 'Got it, thanks for your help with Machine Learning basics.' 
            return: 'Concluding'
    """
    result = llm.invoke(prompt)
    return (
        result + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

In [19]:
@tool #generate_assessment
def generate_assessments(messages: AgentState):
    """A tool that can generate 5 questions that is used to evaluate the user's understanding level of a particular topic"""
    prompt = f"""
        system,
        Analyze the messages:
        {messages}
        Task: 
            Analyze the provided message history and generate 5 insightful questions that can assess the user's
            understanding of the topic. The questions should be tailored to the specific content discussed in the 
            messages, incorporating relevant topic data structures where applicable. The question must be simple
            enough to be answered in a single reply for all the questions in the next query.

        Guidelines:
         - Relevance: Ensure the questions directly relate to the key points or themes addressed in the message history.
         - Clarity: Frame the questions in a clear and concise manner, avoiding ambiguity or vagueness.
         - Variety: Strive for a mix of question types (e.g., open-ended, multiple-choice, short answer) to assess different aspects of understanding.
         - Difficulty: Consider the user's likely level of knowledge and adjust the difficulty of the questions accordingly.
         - Data Structures: When appropriate, incorporate topic-specific data structures (e.g., graphs, trees, arrays) into the questions to assess the user's ability to apply their knowledge to real-world scenarios.
    
    Note: The specific data structures used in the questions will depend on the topic being discussed in the message history.
    For example, if the topic is graph algorithms, questions might involve concepts like adjacency matrices, adjacency lists, or depth-first search.
    """
    result = llm.invoke(prompt)
    return (
        result + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

In [20]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    # We convert the agent output into a format that is suitable to append to the global state
    if isinstance(result, ToolMessage):
        pass
    else:
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        # Since we have a strict workflow, we can
        # track the sender so we know who to pass to next.
        "sender": name,
    }


In [21]:
convo_state_tracker_agent = create_agent(
    llm,
    [convo_state_tracker],
    system_message= "A tool, that can analyze the current conversation and return from one of the values: ['Initial','Exploring','Probing','Concluding']",
)
convo_state_tracker_node = functools.partial(agent_node, agent=convo_state_tracker_agent, name="convo_state_tracker_agent")

generate_assessments_agent = create_agent(
    llm,
    [generate_assessments],
    system_message="Any charts you display will be visible by the user.",
)
generate_assessments_node = functools.partial(agent_node, agent=generate_assessments_agent, name="generate_assessments")

NameError: name 'create_agent' is not defined

In [ ]:
tools = [convo_state_tracker, generate_assessments]
tool_node = ToolNode(tools)

In [ ]:
def router(state) -> Literal["call_tool", "__end__", "continue"]:
    # This is the router  
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        # The previous agent is invoking a tool
        return "call_tool"
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return "__end__"
    return "continue"

In [ ]:
workflow = StateGraph(AgentState)

workflow.add_node("convo_state_tracker", convo_state_tracker_node)
workflow.add_node("generate_assessments", generate_assessments_node)
workflow.add_node("call_tool", tool_node)

conditional_map = {node:node for node in members}
conditional_map["call_tool"] = "call_tool"
conditional_map["__end__"] = END

workflow.add_conditional_edges(
    "convo_state_tracker",
    router,
    {'continue':'generate_assessments','call_tool': 'call_tool', '__end__': END}
)
workflow.add_conditional_edges(
    "generate_assessments",
    router,
    {'continue':END,'call_tool': 'call_tool', '__end__': END}

)
workflow.add_conditional_edges(
    "call_tool",
    lambda x: x["sender"],
    {node:node for node in members}
)

workflow.add_edge(START, "convo_state_tracker")
graph = workflow.compile()

In [ ]:
# from IPython.display import Image, display

# try:
#     display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
# except Exception:
#     # This requires some extra dependencies and is optional
#     pass

In [ ]:
res = []

for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Hi, I wanna learn Datastructures")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")
        res.append(s)

{'convo_state_tracker': {'messages': [AIMessage(content='Sure, I can help with that. What specific aspect of Datastructures are you interested in?', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 980, 'total_tokens': 1000, 'completion_time': 0.017711488, 'prompt_time': 0.130411703, 'queue_time': 0.0014547659999999962, 'total_time': 0.148123191}, 'model_name': 'llama3-groq-8b-8192-tool-use-preview', 'system_fingerprint': 'fp_260dc69250', 'finish_reason': 'stop', 'logprobs': None}, name='convo_state_tracker_agent', id='run-9970263d-25c6-4cff-a3b1-161056b20106-0', usage_metadata={'input_tokens': 980, 'output_tokens': 20, 'total_tokens': 1000})], 'sender': 'convo_state_tracker_agent'}}
----
{'generate_assessments': {'messages': [AIMessage(content="I'm interested in learning about algorithms and their implementations.", response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 980, 'total_tokens': 992, 'completion_time': 0.010251569, 'prompt_

In [ ]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Count the words, 'Hi, I wanna learn Datastructures'")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'convo_state_tracker': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2yjh', 'function': {'arguments': '{"messages": [{"content": "Hi, I wanna learn Datastructures", "type": "user"}]}', 'name': 'convo_state_tracker'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 986, 'total_tokens': 1031, 'completion_time': 0.040843041, 'prompt_time': 0.130306235, 'queue_time': 0.001110252000000006, 'total_time': 0.171149276}, 'model_name': 'llama3-groq-8b-8192-tool-use-preview', 'system_fingerprint': 'fp_260dc69250', 'finish_reason': 'tool_calls', 'logprobs': None}, name='convo_state_tracker_agent', id='run-c90fa38a-adb6-43b8-a595-a1157a807596-0', tool_calls=[{'name': 'convo_state_tracker', 'args': {'messages': [{'content': 'Hi, I wanna learn Datastructures', 'type': 'user'}]}, 'id': 'call_2yjh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 986, 'output_tokens': 45, 'total_tokens': 1031})], 'sender': 

KeyError: 'convo_state_tracker_agent'